In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Drive'a bağlandıktan sonra:

# 1. Basit listelemek için:
import os

# Belirli bir klasördeki dosyaları listele
documents_path = '/content/drive/MyDrive/Colab Notebooks/rag_datasets_test'  # Ana klasör
files = os.listdir(documents_path)
print("Ana klasördeki dosyalar:")
for file in files:
    print(f"📄 {file}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ana klasördeki dosyalar:
📄 Greenland.txt
📄 Afghanistan.txt
📄 Albania.txt
📄 Algeria.txt
📄 Andorra.txt
📄 Angola.txt
📄 Antigua_and_Barbuda.txt
📄 Argentina.txt
📄 Armenia.txt
📄 Australia.txt
📄 Austria.txt
📄 Azerbaijan.txt
📄 Bahamas.txt
📄 Bahrain.txt
📄 Bangladesh.txt
📄 Barbados.txt
📄 Belarus.txt
📄 Belize.txt
📄 Belgium.txt
📄 Benin.txt
📄 Bolivia.txt
📄 Bhutan.txt
📄 Bosnia_and_Herzegovina.txt
📄 Botswana.txt
📄 Brazil.txt
📄 Brunei.txt
📄 Bulgaria.txt
📄 Burkina_Faso.txt
📄 Burundi.txt
📄 Cabo_Verde.txt
📄 Cambodia.txt
📄 Canada.txt
📄 Cameroon.txt
📄 Central_African_Republic.txt
📄 Chad.txt
📄 China.txt
📄 Chile.txt
📄 Comoros.txt
📄 Colombia.txt
📄 Congo.txt
📄 Costa_Rica.txt
📄 Cuba.txt
📄 Croatia.txt
📄 Cyprus.txt
📄 Czech_Republic.txt
📄 Democratic_Republic_of_the_Congo.txt
📄 Denmark.txt
📄 Djibouti.txt
📄 Dominica.txt
📄 Dominican_Republic.txt
📄 Ecuador.txt
📄 Egypt.txt
📄 El_Salvador.txt
📄

In [ ]:
!pip install pypdf2
import PyPDF2
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_process_documents(directory_path):
    """
    Belirtilen dizindeki PDF ve TXT dosyalarını yükler ve işler
    """
    documents = []

    for filename in os.listdir(directory_path):
        filepath = os.path.join(directory_path, filename)

        # PDF dosyalarını işleme
        if filename.endswith('.pdf'):
            with open(filepath, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text()

                process_text_content(text, filename, documents)

        # TXT dosyalarını işleme
        elif filename.endswith('.txt'):
            with open(filepath, 'r', encoding='utf-8') as file:
                text = file.read()
                process_text_content(text, filename, documents)

    return documents

def process_text_content(text, filename, documents):
    """
    Metin içeriğini işler ve belgelere ekler
    """
    # Metni daha küçük parçalara böl
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_text(text)

    for i, chunk in enumerate(chunks):
        documents.append({
            'id': f"{filename}_chunk{i}",
            'text': chunk,
            'source': filename
        })

# Test için örnek kullanım
documents = load_and_process_documents(documents_path)
print(f"Toplam {len(documents)} metin parçası oluşturuldu.")


Toplam 21049 metin parçası oluşturuldu.


In [ ]:
"""
# !pip install chromadb
from sentence_transformers import SentenceTransformer
import chromadb

# Embedding modelini yükle
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# ChromaDB istemcisini başlat
client = chromadb.Client()
collection = client.create_collection(name="notebook_documents")

# Embedding'leri oluştur ve sakla
def store_embeddings(documents):
    ids = []
    texts = []
    embeddings = []
    metadatas = []

    for doc in documents:
        embedding = embedding_model.encode(doc['text']).tolist()

        ids.append(doc['id'])
        texts.append(doc['text'])
        embeddings.append(embedding)
        metadatas.append({'source': doc['source']})

    # ChromaDB'ye ekle
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=metadatas,
        documents=texts
    )

    print(f"{len(ids)} belge vektör veritabanına eklendi.")

# Embedding'leri sakla
store_embeddings(documents)"""


!pip install chromadb
from sentence_transformers import SentenceTransformer
import chromadb
import os

# Embedding modelini yükle
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
vektor_db_path="/content/drive/MyDrive/Colab Notebooks/vector_db_test"

# ChromaDB istemcisini PERSISTENT modda başlat
client = chromadb.PersistentClient(path=vektor_db_path)

# Koleksiyonu oluştur veya var olanı kullan
collection = client.get_or_create_collection(name="notebook_documents")

# Embedding'leri oluştur ve sakla - BATCH EKLEME
def store_embeddings(documents, batch_size=4000):
    total_documents = len(documents)
    print(f"Toplam {total_documents} belge işleniyor...")

    for i in range(0, total_documents, batch_size):
        end_idx = min(i + batch_size, total_documents)
        batch_docs = documents[i:end_idx]

        ids = []
        texts = []
        embeddings = []
        metadatas = []

        for doc in batch_docs:
            embedding = embedding_model.encode(doc['text']).tolist()

            ids.append(doc['id'])
            texts.append(doc['text'])
            embeddings.append(embedding)
            metadatas.append({'source': doc['source']})

        # ChromaDB'ye batch ekle
        collection.add(
            ids=ids,
            embeddings=embeddings,
            metadatas=metadatas,
            documents=texts
        )

        print(f"Batch {i//batch_size + 1}: {len(ids)} belge eklendi. Toplam: {end_idx}/{total_documents}")

    print(f"Tüm belgeler vektör veritabanına eklendi. Veritabanı {vektor_db_path} dizinine kaydedildi.")

# Daha sonra veritabanını yeniden yüklemek için:
def load_saved_collection():
    client = chromadb.PersistentClient(path=vektor_db_path)
    collection = client.get_collection(name="notebook_documents")
    return collection

# Embedding'leri sakla - ARTIK BATCH'LER HALİNDE
store_embeddings(documents)

# Daha sonra kullanmak için kaydedilmiş koleksiyonu yükleme örneği:
# saved_collection = load_saved_collection()
# results = saved_collection.query(...)

Toplam 21049 belge işleniyor...
Batch 1: 4000 belge eklendi. Toplam: 4000/21049
Batch 2: 4000 belge eklendi. Toplam: 8000/21049
Batch 3: 4000 belge eklendi. Toplam: 12000/21049
Batch 4: 4000 belge eklendi. Toplam: 16000/21049
Batch 5: 4000 belge eklendi. Toplam: 20000/21049
Batch 6: 1049 belge eklendi. Toplam: 21049/21049
Tüm belgeler vektör veritabanına eklendi. Veritabanı /content/drive/MyDrive/Colab Notebooks/vector_db dizinine kaydedildi.


'def retrieve_relevant_documents(query, top_k=5):\n\n    # Sorgu için embedding oluştur\n    query_embedding = embedding_model.encode(query).tolist()\n\n    # En benzer sonuçları sorgula\n    results = collection.query(\n        query_embeddings=[query_embedding],\n        n_results=top_k\n    )\n\n    return results\n\n# Test sorgusu\nquery = "Who Founded Ottoman empire"\nresults = retrieve_relevant_documents(query)\nprint("En ilgili 3 belge:")\nfor i, doc in enumerate(results[\'documents\'][0][:3]):\n    print(f"{i+1}. {doc[:100]}...")'

In [ ]:

vektor_db_path="/content/drive/MyDrive/Colab Notebooks/vector_db"


client = chromadb.PersistentClient(path=vektor_db_path)
collection = client.get_collection(name="notebook_documents")

def retrieve_relevant_documents(query, top_k=5):
    """
    Sorguya en benzer belgeleri getir
    """
    # Sorgu için embedding oluştur
    query_embedding = embedding_model.encode(query).tolist()

    # En benzer sonuçları sorgula
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    return results

# Test sorgusu
query = "Who Founded Ottoman empire"
results = retrieve_relevant_documents(query)
print("En ilgili 3 belge:")
for i, (doc, metadata) in enumerate(zip(results['documents'][0][:3], results['metadatas'][0][:3])):
    print(f"{i+1}. {doc[:100]}... (Source: {metadata['source']})")

En ilgili 3 belge:
1. Ottoman Empire
Based around Söğüt, Ottoman Beylik was founded by Osman I in the early 14th century. ... (Source: Turkey.txt)
2. The Ottoman Empire was a global power during the reigns of Selim I and Suleiman the Magnificent. In ... (Source: Turkey.txt)
3. First inhabited by modern humans during the Late Paleolithic, present-day Turkey was home to various... (Source: Turkey.txt)
